--- Day 3: Mull It Over ---
"Our computers are having issues, so I have no idea if we have any Chief Historians in stock! You're welcome to check the warehouse, though," says the mildly flustered shopkeeper at the North Pole Toboggan Rental Shop. The Historians head out to take a look.

The shopkeeper turns to you. "Any chance you can see why our computers are having issues again?"

The computer appears to be trying to run a program, but its memory (your puzzle input) is corrupted. All of the instructions have been jumbled up!

It seems like the goal of the program is just to multiply some numbers. It does that with instructions like mul(X,Y), where X and Y are each 1-3 digit numbers. For instance, mul(44,46) multiplies 44 by 46 to get a result of 2024. Similarly, mul(123,4) would multiply 123 by 4.

However, because the program's memory has been corrupted, there are also many invalid characters that should be ignored, even if they look like part of a mul instruction. Sequences like mul(4*, mul(6,9!, ?(12,34), or mul ( 2 , 4 ) do nothing.

For example, consider the following section of corrupted memory:

xmul(2,4)%&mul[3,7]!@^do_not_mul(5,5)+mul(32,64]then(mul(11,8)mul(8,5))
Only the four highlighted sections are real mul instructions. Adding up the result of each instruction produces 161 (24 + 55 + 118 + 85).

Scan the corrupted memory for uncorrupted mul instructions. What do you get if you add up all of the results of the multiplications?



In [3]:
import re

def scan_memory_with_conditionals(text):
    """
    Scan corrupted memory text for valid mul instructions, respecting do() and don't() conditionals.
    
    Args:
        text (str): The corrupted memory text to scan
        
    Returns:
        int: Sum of all valid and enabled multiplication results
    """
    # Regular expressions for instructions
    mul_pattern = r'mul\((\d{1,3}),(\d{1,3})\)'
    do_pattern = r'do\(\)'
    dont_pattern = r'don\'t\(\)'
    
    # Get all instructions with their positions
    instructions = []
    
    # Find all mul instructions
    for match in re.finditer(mul_pattern, text):
        instructions.append({
            'type': 'mul',
            'pos': match.start(),
            'x': int(match.group(1)),
            'y': int(match.group(2))
        })
    
    # Find all do() instructions
    for match in re.finditer(do_pattern, text):
        instructions.append({
            'type': 'do',
            'pos': match.start()
        })
    
    # Find all don't() instructions
    for match in re.finditer(dont_pattern, text):
        instructions.append({
            'type': 'dont',
            'pos': match.start()
        })
    
    # Sort instructions by position
    instructions.sort(key=lambda x: x['pos'])
    
    # Process instructions
    total = 0
    enabled = True  # Initially enabled
    calculations = []
    
    for instruction in instructions:
        if instruction['type'] == 'do':
            enabled = True
        elif instruction['type'] == 'dont':
            enabled = False
        elif instruction['type'] == 'mul' and enabled:
            x = instruction['x']
            y = instruction['y']
            result = x * y
            calculations.append(f"{x} * {y} = {result} (enabled: {enabled})")
            total += result
        elif instruction['type'] == 'mul':
            x = instruction['x']
            y = instruction['y']
            calculations.append(f"{x} * {y} = {x * y} (enabled: {enabled}, not counted)")
    
    return total, calculations

def process_file(filename):
    """
    Read and process the input file.
    
    Args:
        filename (str): Path to the input file
        
    Returns:
        int: Sum of all enabled multiplication results
    """
    try:
        with open(filename, 'r') as file:
            content = file.read()
        total, calculations = scan_memory_with_conditionals(content)
        
        # Print each calculation for verification
        print("Individual calculations:")
        for calc in calculations:
            print(calc)
            
        print(f"\nTotal sum: {total}")
        return total
    except FileNotFoundError:
        print(f"Error: Could not find file {filename}")
        return None
    except Exception as e:
        print(f"Error processing file: {e}")
        return None

filename = r"input.txt" 

result = process_file(filename)

if result is not None:
    print(f"\nFinal answer: {result}")

Individual calculations:
747 * 16 = 11952 (enabled: True)
354 * 748 = 264792 (enabled: True)
29 * 805 = 23345 (enabled: True)
480 * 119 = 57120 (enabled: True)
685 * 393 = 269205 (enabled: True)
376 * 146 = 54896 (enabled: True)
735 * 916 = 673260 (enabled: True)
321 * 623 = 199983 (enabled: True)
363 * 643 = 233409 (enabled: True)
360 * 266 = 95760 (enabled: True)
95 * 167 = 15865 (enabled: True)
802 * 119 = 95438 (enabled: True)
147 * 169 = 24843 (enabled: True)
488 * 194 = 94672 (enabled: True)
540 * 154 = 83160 (enabled: True)
8 * 750 = 6000 (enabled: True)
140 * 841 = 117740 (enabled: True)
428 * 793 = 339404 (enabled: True)
156 * 996 = 155376 (enabled: True)
976 * 569 = 555344 (enabled: True)
426 * 703 = 299478 (enabled: True)
948 * 128 = 121344 (enabled: True)
477 * 567 = 270459 (enabled: True)
182 * 79 = 14378 (enabled: True)
203 * 707 = 143521 (enabled: True)
186 * 170 = 31620 (enabled: True)
130 * 392 = 50960 (enabled: True)
563 * 902 = 507826 (enabled: True)
953 * 129 = 1229

--- Part Two ---
As you scan through the corrupted memory, you notice that some of the conditional statements are also still intact. If you handle some of the uncorrupted conditional statements in the program, you might be able to get an even more accurate result.

There are two new instructions you'll need to handle:

The do() instruction enables future mul instructions.
The don't() instruction disables future mul instructions.
Only the most recent do() or don't() instruction applies. At the beginning of the program, mul instructions are enabled.

For example:

xmul(2,4)&mul[3,7]!^don't()_mul(5,5)+mul(32,64](mul(11,8)undo()?mul(8,5))
This corrupted memory is similar to the example from before, but this time the mul(5,5) and mul(11,8) instructions are disabled because there is a don't() instruction before them. The other mul instructions function normally, including the one at the end that gets re-enabled by a do() instruction.

This time, the sum of the results is 48 (24 + 85).

Handle the new instructions; what do you get if you add up all of the results of just the enabled multiplications?




In [5]:
import re

# Open and read the input file
with open('input.txt', 'r') as file:
    data = file.read()

# Regular expressions to match instructions
mul_pattern = r"mul\((\d{1,3}),(\d{1,3})\)"  # Matches mul(X,Y)
do_pattern = r"do\(\)"                        # Matches do()
dont_pattern = r"don't\(\)"                   # Matches don't()

# Find all matches in the input, preserving order
instructions = re.findall(f"({do_pattern})|({dont_pattern})|{mul_pattern}", data)

# Variable to track if mul instructions are enabled
enabled = True

# Variable to store the total sum
total = 0

# Process each instruction
for instruction in instructions:
    if instruction[0]:  # Matches do()
        enabled = True
    elif instruction[1]:  # Matches don't()
        enabled = False
    elif instruction[2] and instruction[3] and enabled:  # Matches mul(X,Y) if enabled
        x, y = map(int, instruction[2:4])  # Extract numbers from the mul() match
        total += x * y  # Add their product to the total

# Print the result
print(total)

77055967
